# Содержание
* [Определение констант, необходимых функций](#id0)
* [Сбор и подготовка размеченного файла](#id1)
* [Загрузка и подготовка данных](#id2)
* [Оценка точности](#id3)

In [5]:
import pandas as pd
import numpy as np
from sklearn import metrics
from tqdm import tqdm
from IPython.display import display
from datetime import datetime
import gc
from typing import Optional, List, Union

gc.enable()
pd.set_option("future.no_silent_downcasting", True)

# Определение констант, необходимых функций <a id="id0"></a>

In [3]:
FILE_PREDICTIONS = "data/structured_messages-8.xlsx"
FILES_GROUND_TRUTH = ("data/train_data.xlsx", "data/train_data2.xlsx", "data/70-109.xlsx")
OUTPUT_FILE = "1.xlsx"

CURRENT_YEAR = current_year = datetime.now().year
DATE_FORMATS = [
    "%d.%m", "%d/%m", "%d.%m.%Y", "%d/%m/%Y", "%d.%m.%y", "%d/%m/%y",
    "%Y-%m-%d"
]

COLUMNS = [
    "msg_id", "Дата", "Подразделение", "Операция", "Культура",
    "За день, га", "С начала операции, га", "Вал за день, ц",
    "Вал с начала, ц"
]
TEXT_COLS = ["Дата", "Подразделение", "Операция", "Культура"]
NUM_COLS = ["За день, га", "С начала операции, га", "Вал за день, ц", "Вал с начала, ц"]

In [3]:
def load_data(path: str) -> pd.DataFrame:
    """
    Загружает данные из Excel-файла, выполняет предварительную обработку и проверку столбцов.

    Функция читает Excel-файл по указанному пути, загружая столбец "Дата" как строковый тип.
    Если в данных присутствуют столбцы "text" или "message", они удаляются.
    После этого имена столбцов заменяются на значения из глобальной переменной COLUMNS.

    :param path: str
        Путь к Excel-файлу для загрузки.
    :return: pd.DataFrame
        Обработанный DataFrame с переименованными столбцами.
    
    :raises FileNotFoundError:
        Если файл по указанному пути не найден.
    :raises ValueError:
        Если количество столбцов в файле не совпадает с длиной COLUMNS.
    """
    df = pd.read_excel(path, dtype={"Дата": str})
    if "text" in df.columns:
        df.drop("text", axis=1, inplace=True)
    if "message" in df.columns:
        df.drop("message", axis=1, inplace=True)
    df.columns = COLUMNS
    return df

In [4]:
def parse_datе(date: str) -> Optional[str]:
    """
    Парсит строку с датой и преобразует её в формат "дд.мм.гггг".

    Функция принимает строку с датой, обрезает время (если есть), удаляет лишние пробелы и завершающую точку.
    Затем пытается распарсить дату по списку форматов из глобальной переменной DATE_FORMATS.
    Если формат совпадает с одним из первых двух, год заменяется на текущий (CURRENT_YEAR).
    Возвращает строку с датой в формате "дд.мм.гггг".

    :param date: str
        Исходная строка с датой, возможно содержащая время и дополнительные символы.
    :return: str или None
        Отформатированная дата в виде строки "дд.мм.гггг" или None, если парсинг не удался.
    """
    date = date.split(' ')[0]
    date = date.strip().rstrip('.')
    for i, fmt in enumerate(DATE_FORMATS):
        try:
            dt = datetime.strptime(date, fmt)
            if i < 2:
                dt = dt.replace(year=CURRENT_YEAR)
            dt = dt.strftime("%d.%m.%Y")
            return dt
        except Exception as e:
            pass

In [5]:
def calc_acc(
    y_true: List[Union[str, int, float]],
    y_pred: y_true: List[Union[str, int, float]]
) -> float:
    """
    Вычисляет точность (accuracy) между истинными и предсказанными значениями с учетом частичного совпадения строк.

    Функция принимает два списка или последовательности одинаковой длины: истинные значения (`y_true`) и предсказанные (`y_pred`).
    Точность считается как доля совпадений по элементам:
    - Полное совпадение значений увеличивает счетчик точных совпадений.
    - Если оба значения — строки, и истинное значение начинается с предсказанного, это также считается совпадением.

    :param y_true: list
        Список истинных значений.
    :param y_pred: list
        Список предсказанных значений.
    :return: float
        Отношение количества совпадений к общему числу элементов, значение в диапазоне [0, 1].

    :raises AssertionError:
        Если длины `y_true` и `y_pred` не совпадают.
    """
    
    assert len(y_true) == len(y_pred), f"true={len(y_true)}, pred={len(y_pred)}"

    acc = 0.0
    for val in zip(y_true, y_pred):
        true_value, pred_value = val[0], val[1]
        if true_value == pred_value:
            acc += 1
        else:
            if isinstance(true_value, str) and isinstance(pred_value, str):
                if true_value.startswith(pred_value):
                    acc += 1
    return acc / len(y_true)

# Сбор и подготовка размеченного файла <a id="id1"></a>

Здесь происходит объединение таблиц и синхронизация id сообщений, т.к. разметка производилась каждым членом команды

In [6]:
# dfs = list(map(load_data, FILES_GROUND_TRUTH))
# dfs[-1]["msg_id"] += 69
# pd.concat(dfs, axis=0).to_excel(OUTPUT_FILE, index=False)

Ячейка запускается один раз, потом комментируется.  
Её основная цель - это создать единый размеченный файл

# Загрузка и подготовка данных <a id="id2"></a>

Определим какие операции не оцениваются в рамках хакатона, а также небольшое отображение

In [56]:
DROP_OP = [
    "боронование довсходовое",
    "выкашивание отцовских форм подсолнечник",
    "тестовая операция",
    "средства защиты растений",
    "затравка мышевидных грызунов"
]

MAP = {
    "внесение противозлакового гербицида": "гербицидная обработка",
    "посев": "сев",
    "вспашка": "пахота",
    "сплошная культивация": "культивация",
    "выравнивание": "выравнивание зяби",
    "химическая прополка": "гербицидная обработка"
}

Загрузим данные

In [7]:
train = pd.read_excel(OUTPUT_FILE)
pred = pd.read_excel(FILE_PREDICTIONS)

assert train.shape[1] == pred.shape[1]

pred.columns = COLUMNS

Проверим соглассованность id сообщений размеченных и предсказанных данных  

In [58]:
train.msg_id.max(), pred.msg_id.max()

(105, 105)

Посмотрим так на размерности данных

In [59]:
train.shape, pred.shape

((355, 9), (382, 9))

Кол-во строк не совпадает. Это означает, что где-то алгоритм экстракции информации нашёл или больше, или меньше релевантной информации, а также сюда попали строки с операциями, которые не оцениваются в рамках хакатона, которые, поэтому, надо удалить

Получим уникальные id сообщений

In [9]:
MSG_IDS = pred["msg_id"].unique()

Выполним нормализацию строк из текстовых столбцов

In [61]:
for txt_col in TEXT_COLS:
    train[txt_col] = train[txt_col].str.lower()
    train[txt_col] = train[txt_col].str.replace('ё', 'е')
    
    pred[txt_col] = pred[txt_col].str.lower()
    pred[txt_col] = pred[txt_col].str.replace('ё', 'е')

Провалидирум целовые столбцы на соответствие их числовым типам данных

In [62]:
for num_col in NUM_COLS:
    dtype = train[num_col].dtype.name
    if ("int" in dtype) or ("float" in dtype):
        continue

In [63]:
for num_col in NUM_COLS:
    dtype = pred[num_col].dtype.name
    if ("int" in dtype) or ("float" in dtype):
        continue
    pred[num_col] = pd.to_numeric(pred[num_col], errors="coerce")

Распарсим дату и приведём к единому формату

In [64]:
for i, val in enumerate(train["Дата"]):
    if not pd.isna(val):
        train.loc[i, "Дата"] = parse_datе(str(val))

In [65]:
for i, val in enumerate(pred["Дата"]):
    if not pd.isna(val):
        pred.loc[i, "Дата"] = parse_datе(str(val))

Посмотрим разницу операций между двумя файлами - с предсказаниями и с размеченными данными

In [66]:
set(pred["Операция"].dropna().unique()).difference(set(train["Операция"].unique()))

{'боронование довсходовое',
 'вспашка',
 'выкашивание отцовских форм подсолнечник',
 'выравнивание',
 'выравнивание многолетних трав',
 'затравка мышевидных грызунов',
 'прикатывание',
 'сплошная культивация',
 'средства защиты растений',
 'химическая прополка'}

Удалим те операции, которые не оцениваются в рамках хакатона

In [67]:
pred.drop(pred[pred["Операция"].isin(DROP_OP)].index, axis=0, inplace=True)
pred.reset_index(drop=True, inplace=True)

Выполним приведение в соответствие оставшихся операций, используя отображение, определённое в самом начале раздела

In [68]:
pred["Операция"] = pred["Операция"].apply(lambda x: MAP.get(x, x))

Выполним проверку на наличие пропусков по операции. Если такие строки имеются, то их удалим

In [69]:
nan_operation = pred["Операция"].isna().sum()
if nan_operation != 0:
    display(pred[pred["Операция"].isna()])
    pred.drop(pred[pred["Операция"].isna()].index, axis=0, inplace=True)
    pred.reset_index(drop=True, inplace=True)

,msg_id,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
320,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Выполним замену кода подразделения, а также культуры

In [70]:
pred["Подразделение"] = pred["Подразделение"].apply(lambda x: "аор" if x.startswith("отд") else x)

In [71]:
pred["Культура"] = pred["Культура"].apply(lambda x: "озимые культуры" if x == "озимые" else x)

Выполним проверку на наличие пропусков по культуре. Операции с пустой культурой удалим, т.к. эти операции не несут в себе полезной информации

In [72]:
nan_cultutre = pred["Культура"].isna().sum()
if nan_cultutre != 0:
    display(pred[pred["Культура"].isna()])
    pred.drop(pred[pred["Культура"].isna()].index, axis=0, inplace=True)
    pred.reset_index(drop=True, inplace=True)

,msg_id,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
214,60,NaN,аор,предпосевная культивация,NaN,177.0,396.0,NaN,NaN
298,87,NaN,аор,предпосевная культивация,NaN,NaN,NaN,NaN,NaN
299,87,NaN,аор,выравнивание зяби,NaN,180.0,1430.0,NaN,NaN
301,88,NaN,аор,подкормка,NaN,218.0,NaN,NaN,NaN
302,88,NaN,аор,культивация,NaN,196.0,1626.0,NaN,NaN


Оценим теперь размерности данных

In [74]:
train.shape, pred.shape

((355, 9), (361, 9))

Выполнив очистку от "шумовых" данных, размерности данных почти удалось свести

Для удобства оценки точности работы алгоритма экстракции информации заполним все пропуски значением "-1"

In [75]:
train = train.fillna(-1)
pred = pred.fillna(-1)

Проверим, что пропусков больше не осталось

In [76]:
train.isna().sum().sum(), pred.isna().sum().sum()

(0, 0)

В заключении оценим, имеется ли разница по операциям

In [77]:
set(pred["Операция"].dropna().str.lower().unique()).difference(
    set(train["Операция"].unique())
)

{'выравнивание многолетних трав', 'прикатывание'}

Разница осталась. По первому элементу - данная операция не входила в список рассматриваемых во время хакатона. По второй операции - из-за наличия операциия "прикатывание посевов" можно оставить, т.к. префиксы совпадют; семантика операций отличаться не будет  

Решено было оставить эти операции

# Оценка точности <a id="id3"></a>

Выполним оценку точности работы алгоритма экстракции информации

In [83]:
scores = []
cnt = 0
for msg_id in MSG_IDS:
    y_true, y_pred = [], []
    tmp_true = train[train["msg_id"].values == msg_id].fillna(-1).values[:, 1:]
    tmp_pred = pred[pred["msg_id"].values == msg_id].fillna(-1).values[:, 1:]
    if tmp_true.shape[0] == tmp_pred.shape[0]:
        y_true.extend(tmp_true.ravel().tolist())
        y_pred.extend(tmp_pred.ravel().tolist())
        score = calc_acc(y_true, y_pred)
        scores.append(score)
    else:
        t_size, p_size = tmp_true.shape[0], tmp_pred.shape[0]
        if (t_size == 1) and (p_size > 0):
            scores.append(
                calc_acc(tmp_true.ravel().tolist(), tmp_pred[:1].ravel().tolist())
            )
            continue
        cnt += 1

        print(msg_id, t_size, p_size)

sum(scores) / len(scores), len(scores), cnt

19 4 5
29 6 5
32 7 8
37 10 9
45 4 6
60 3 2
72 4 2
78 4 3
81 8 7
82 8 6
83 5 3
88 2 1
93 1 0
99 5 4


(0.9534220798826776, 92, 14)

Точность алгоритма - **95.34%**

Матрица, которая выводится перед точность - это информация, которая нужна была для оценки работы алгоритма определния точности